In [1]:
%cd /home/jaeheonshim/music-vibes

/home/jaeheonshim/music-vibes


In [ ]:
import torch
import numpy as np
import pandas as pd
from torch import nn
from torch.optim.lr_scheduler import CosineAnnealingLR
import torch.nn.functional as F
from torch.utils.data import random_split, DataLoader, Subset
import librosa
from tqdm import tqdm
from torchvggish import vggish_input

from vibenet import labels
from vibenet.dataset import FMAVGGishDataset
from vibenet.models.teacher import VGGishLSTM
from vibenet.utils import Mixup, do_mixup

In [4]:
dataset = FMAVGGishDataset('data/preprocessed/vggish')
# dataset = Subset(dataset, list(range(50)))

N = len(dataset)
print(N)
train_size = int(N * 0.8)
train_ds, test_ds = random_split(dataset, [train_size, N - train_size], generator=torch.Generator().manual_seed(42))
train_dl = DataLoader(train_ds, batch_size=64, shuffle=True)
test_dl = DataLoader(test_ds, batch_size=64, shuffle=False)

13122


In [5]:
device = 'cuda'

model = VGGishLSTM()
model = model.to(device)

NUM_EPOCHS = 200

optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
scheduler = CosineAnnealingLR(optimizer, T_max=NUM_EPOCHS, eta_min=1e-5)
mixup = Mixup(0.2)

In [ ]:
best_loss = 1

for epoch in range(NUM_EPOCHS):
    scheduler.step()

    print(f"Epoch {epoch+1}:")
    model.train()
    
    train_losses = []

    with tqdm(train_dl, desc='Training') as pbar:
        for data, label in pbar:
            data, label = data.to(device).float(), label.to(device).float()
            optimizer.zero_grad()

            if data.shape[0] % 2 != 0:
                data = data.expand(2, -1, -1, -1)
                label = label.expand(2, -1)

            mixup_lambda = torch.from_numpy(mixup.get_lambda(data.shape[0]).astype(np.float32)).to(device)
            label = do_mixup(label, mixup_lambda)

            with torch.autocast(device_type="cuda"):
                pred = model(data, mixup_lambda)
                loss = F.l1_loss(pred, label)

            loss.backward()
            optimizer.step()

            train_losses.append(loss.item())
            train_loss = np.mean(train_losses)

            pbar.set_postfix({'loss': f"{train_loss:.4f}", 'lr': f"{scheduler.get_last_lr()[0]}"})

    model.eval()

    eval_losses = []
    label_maes = []

    with tqdm(test_dl, desc='Validation') as pbar:
        with torch.inference_mode():
            for data, label in pbar:
                data, label = data.to(device).float(), label.to(device).float()

                pred = model(data)
                loss = F.l1_loss(pred, label)

                eval_losses.append(loss.item())
                eval_loss = np.mean(eval_losses)

                mae = torch.mean(torch.abs(pred - label), dim=0) # for per-metric MAE
                label_maes.append(mae.cpu().numpy())

                pbar.set_postfix({'loss': f"{eval_loss:.4f}"})
    
    eval_loss = np.mean(eval_losses)
    print("Validation loss:", eval_loss)

    label_mae = np.mean(label_maes, axis=0)
    print("MAE per label:")
    for i, label in enumerate(labels):
        print(f"\t{label:<20}{label_mae[i]:.4f}")

    if eval_loss < best_loss:
        best_loss = eval_loss
        torch.save(model.state_dict(), 'best_model.pt')
        print('Saved new best model')

/home/jaeheonshim/music-vibes/venv/lib64/python3.11/site-packages/torch/optim/lr_scheduler.py:182: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn(


Epoch 1:


Training:   0%|          | 0/165 [00:00<?, ?it/s]/home/jaeheonshim/music-vibes/venv/lib64/python3.11/site-packages/torch/utils/data/_utils/collate.py:285: UserWarning: The given NumPy array is not writable, and PyTorch does not support non-writable tensors. This means writing to this tensor will result in undefined behavior. You may want to copy the array to protect its data or make it writable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at /pytorch/torch/csrc/utils/tensor_numpy.cpp:203.)
  return collate([torch.as_tensor(b) for b in batch], collate_fn_map=collate_fn_map)
Validation: 100%|██████████| 42/42 [00:04<00:00,  9.54it/s, loss=0.1751]


Validation loss: 0.1751357778197243
MAE per label:
	acousticness        0.2671
	danceability        0.1322
	energy              0.1565
	instrumentalness    0.2750
	liveness            0.1037
	speechiness         0.0630
	valence             0.2284
Saved new best model
Epoch 2:


Validation: 100%|██████████| 42/42 [00:04<00:00, 10.05it/s, loss=0.1659]


...

Validation loss: 0.1312478000209445
MAE per label:
	acousticness        0.1924
	danceability        0.1017
	energy              0.1177
	instrumentalness    0.1926
	liveness            0.0922
	speechiness         0.0481
	valence             0.1742
Epoch 86:


Validation: 100%|██████████| 42/42 [00:04<00:00,  9.21it/s, loss=0.1307]


Validation loss: 0.1307020710692519
MAE per label:
	acousticness        0.1935
	danceability        0.0979
	energy              0.1176
	instrumentalness    0.1908
	liveness            0.0922
	speechiness         0.0486
	valence             0.1743
Epoch 87:


Training:  18%|█▊        | 30/165 [00:04<00:19,  7.07it/s, loss=0.1227, lr=0.0006053797112014735]


KeyboardInterrupt: 

In [6]:
model.load_state_dict(torch.load('best_model.pt'))

y, sr = librosa.load("Dancing Queen.flac", sr=16000)
log_mel = vggish_input.waveform_to_examples(y, sample_rate=16000).to(device).float()
log_mel = log_mel.squeeze(1)
log_mel = log_mel.unsqueeze(0)

model.eval()

with torch.inference_mode():
    with torch.autocast(device_type="cuda"):
        pred = model(log_mel)

pred = pred[0].cpu().numpy()
for i, label in enumerate(labels):
    print(f"{label:<20}{pred[i]:.4f}")

acousticness        0.4377
danceability        0.5791
energy              0.7266
instrumentalness    0.5913
liveness            0.1249
speechiness         0.0463
valence             0.7236
